In [1]:
import numpy as np 
from matplotlib import pyplot as plt 

plt.rcParams.update({
    "text.usetex": True,            # Use LaTeX for all text
    "font.family": "serif",         # Use serif font
    "font.serif": ["Computer Modern Roman"],  # LaTeX default
    "axes.labelsize": 14,
    "font.size": 16,
    "legend.fontsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
}) 

import os 
from pathlib import Path 

import re 


In [2]:
FOLDER = "MNIST_STRATIFIED_CLASSIFIERS_MADGAN_NEW"

DATASET, _, _, TYPE, _ = FOLDER.split('_')

metrics = ["val_accuracy", "val_f1_score", "val_loss"]   

base_path = Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/notebooks")
strat_exp_path = base_path / 'experiments' / FOLDER 

experiments = os.listdir(strat_exp_path)


In [3]:
cmap = plt.get_cmap('tab10')  # good for distinct colors (up to 10)
colors = [cmap(i) for i in range(10)] + [(0.5, 0.5, 0.5, 1.0)]  # Add gray for the 11th color

color_dict_by_generator = {str(i): colors[i] for i in range(11)}


def extract_info_from_experiment_name(exp: str) -> dict:
    """
    Extracts metadata (dataset, generator info, image counts) from an experiment name string.
    Handles missing generator information gracefully by assigning default integer values.

    Args:
        exp (str): The experiment name string.

    Returns:
        dict: A dictionary containing the extracted metadata, with values converted to appropriate types.
              Returns an empty dictionary if the name doesn't match the expected pattern.
    """
    # General pattern supporting both BASE and MADGAN variants
    # Made generator parts optional and non-greedy where appropriate
    pattern = (
        r".*?_" # Match anything non-greedily up to the first underscore before dataset
        r"(?P<dataset>[A-Z0-9]+)_+" # Dataset name (allow numbers like CIFAR10)
        # Optional group for generator info - make the whole thing optional
        r"(?:" # Start non-capturing group for optional generator parts
            # Optional used_generator part (non-greedy match before it)
            r"(?:.*?" # Non-greedy match any char
              r"used_generator_(?P<used_gen>\d+)_+" # Capture used_gen
            r")?" # End optional used_generator part
            # Optional trained_generators part (non-greedy match before it)
            r"(?:.*?"
              r"trained_generators_(?P<n_gen>\d+)_+" # Capture n_gen
            r")?" # End optional trained_generators part
        r")?" # End non-capturing group for optional generator parts
        # Mandatory image count part (allow non-greedy match before it)
        r".*?"
        r"images_real_(?P<n_real>\d+)_+" # Capture n_real
        r"gen_(?P<n_fake>\d+)" # Capture n_fake (allow pattern to end here or have more chars)
        r".*$" # Match any remaining characters to the end of the string
    )


    match = re.search(pattern, exp)
    if not match:
        print(f"ALARM: Could not parse experiment name: '{exp}'")
        # You might want to return defaults or raise an error depending on requirements
        return {'dataset': 'UNKNOWN', 'used_gen': 0, 'n_gen': 0, 'n_real': 0, 'n_fake': 0} # Example defaults

    d = match.groupdict()

    # Safely convert captured groups to integers, providing a default value (e.g., 0)
    # if the optional groups (used_gen, n_gen) were not matched (resulting in None).
    # The 'or 0' idiom handles None gracefully for int conversion.
    d['used_gen'] = int(d['used_gen'] or 0)
    d['n_gen'] = int(d['n_gen'] or 0)

    # n_real and n_fake should ideally always be matched if the overall pattern matches,
    # but add checks/defaults for robustness in case the regex fails unexpectedly.
    d['n_real'] = int(d['n_real'] or 0)
    d['n_fake'] = int(d['n_fake'] or 0)

    # Note: The keys like 'dataset' remain strings. Only numeric parts are converted.

    return d

def sort_dict_based_on_n_real_images(d: dict, reverse=True) -> dict: 
    return dict(sorted(
        d.items(),
        key=lambda x: int(re.search(r'images_real_(\d+)', x[0]).group(1)), 
        reverse=reverse
    ))


def sort_print_metrics(line):
    import re
    # Extract G_{X_Y}
    g_match = re.search(r'G_\{(\d+)_(\d+)\}', line)
    # Extract real and fake image counts (3rd and 4th fields)
    nums = re.findall(r'&\s*(\d+)', line)

    if g_match and len(nums) >= 2:
        g_x, g_y = map(int, g_match.groups())
        real_images = int(nums[0])
        fake_images = int(nums[1])
        return (g_x, g_y, real_images, fake_images)

    return (float('inf'), float('inf'), float('inf'), float('inf'))  # fallback




In [4]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import warnings

metric_map = {
    "val_f1_score": "Val F1",
    "val_loss": "Val Loss",
    "val_accuracy": "Val Acc."
}

# Assume sort_print_metrics function exists if it was used before,
# otherwise remove or comment out the sorting line if no longer needed.
# Example dummy function if needed elsewhere (though not for the new table logic):
# def sort_print_metrics(metric_line):
#     # Implement sorting logic if needed for other parts,
#     # based on e.g., G index extracted from the line
#     try:
#         # Example: sort by used_gen value if present
#         gen_part = metric_line.split('\\(G_{')[1].split('}')[0]
#         n_gen, used_gen = map(int, gen_part.split(','))
#         return (n_gen, used_gen)
#     except:
#         return (float('inf'), float('inf')) # Fallback sorting

def plot_history_strat_classifiers(histories: dict, meta_info: dict, save_path: Path, target_gen, METRIC, title_prefix, title_suffix, show: bool = False, verbose: bool = False, print_metrics: bool = False) -> None:
    histories_copy = histories.copy()
    meta_info_copy = meta_info.copy()
    plt.figure(figsize=(10, 6))

    # --- Baseline Identification (remains the same) ---
    baseline_exp_key = None
    baseline_criteria_real = 5000
    baseline_criteria_fake = 0
    potential_baselines = []

    for exp_key, meta in meta_info_copy.items():
        if exp_key not in histories_copy:
            continue
        try:
            n_real = int(meta.get('n_real', -1))
            n_fake = int(meta.get('n_fake', -1))
            if n_real == baseline_criteria_real and n_fake == baseline_criteria_fake:
                potential_baselines.append(exp_key)
        except (ValueError, TypeError) as e:
            if verbose:
                warnings.warn(f"Could not parse n_real/n_fake for experiment '{exp_key}'. Skipping for baseline check. Error: {e}")
            continue

    if len(potential_baselines) >= 1:
        baseline_exp_key = potential_baselines[0]
        if verbose:
            print(f"Identified baseline experiment: '{baseline_exp_key}' (n_real={baseline_criteria_real}, n_fake={baseline_criteria_fake})")
        if len(potential_baselines) > 1:
            if verbose:
                warnings.warn(f"Multiple experiments match baseline criteria: {potential_baselines}. Using '{baseline_exp_key}' and removing others from this plot.")
            # Remove duplicates from copies used for plotting/aggregation
            for key_to_remove in potential_baselines:
                if key_to_remove != baseline_exp_key:
                    histories_copy.pop(key_to_remove, None)
                    meta_info_copy.pop(key_to_remove, None)
                    if verbose:
                        print(f"--> Removed duplicate baseline '{key_to_remove}' from consideration for this plot.")
    else:
        if verbose:
            print(f"Warning: No baseline experiment found matching criteria (n_real={baseline_criteria_real}, n_fake={baseline_criteria_fake}). Baseline will not be plotted.")
    # --- End Baseline Identification ---


    best_run_key = None
    best_run_hist = None
    worst_run_key = None
    worst_run_hist = None
    lower_is_better = 'loss' in METRIC.lower()

    # Initialize best/worst values
    if lower_is_better:
        best_val = float('inf')
        worst_val = float('-inf')
    else:
        best_val = float('-inf')
        worst_val = float('inf')

    all_vals = [] # For calculating epoch-wise avg/median plot lines
    final_vals_list = [] # For calculating avg of final values for the table

    # --- Loop through experiments to find best/worst and plot faint lines ---
    for exp, hist in histories_copy.items():
        if exp == baseline_exp_key:
            continue # Skip baseline for best/worst/avg calculations and faint plots

        # --- Validate data for this experiment (remains the same) ---
        if METRIC not in hist:
            if verbose: print(f"Warning: Metric '{METRIC}' not found in history for experiment '{exp}'. Skipping.")
            continue
        if not hist[METRIC] or not hasattr(hist[METRIC], '__len__') or len(hist[METRIC]) == 0:
            if verbose: print(f"Warning: Metric list for '{METRIC}' is empty or invalid for experiment '{exp}'. Skipping.")
            continue
        if not all(isinstance(x, (int, float)) for x in hist[METRIC]):
             if verbose: print(f"Warning: Non-numeric data found in metric list for '{METRIC}' in experiment '{exp}'. Skipping.")
             continue
        if exp not in meta_info_copy:
            if verbose: print(f"Warning: Meta info not found for experiment '{exp}' in copy. Skipping plot details.")
            # Allow plotting faint line but skip best/worst update if meta is missing? Or skip entirely? Let's skip entirely.
            continue
        # --- End Validation ---


        # --- Plot faint line for current experiment (remains the same) ---
        try:
            current_meta = meta_info_copy[exp] # Meta exists based on check above
            n_gen = current_meta.get('n_gen', 'N/A')
            used_gen = current_meta.get('used_gen', 'N/A')
            n_gen_int = int(n_gen)
            used_gen_int = int(used_gen)
            # Example coloring (can be simplified to just faint grey)
            # color_val = np.clip((used_gen_int + 1) / n_gen_int, 0, 1) if n_gen_int > 0 else 0.5
            # run_color = (0.1, 0.2, color_val * 0.8 + 0.2, 0.25)
            col = (0.6, 0.6, 0.6, 0.3) # Faint grey
            plt.plot(hist[METRIC], color=col)
        except Exception as e:
            if verbose: print(f"Warning: An unexpected error occurred plotting run {exp}: {e}. Using default color.")
            plt.plot(hist[METRIC], color=(0.6, 0.6, 0.6, 0.3)) # Default faint grey
        # --- End Plot faint line ---


        # --- Update best/worst and collect values (remains the same logic) ---
        all_vals.append(hist[METRIC]) # For epoch-wise plot avg/median
        current_final_val = hist[METRIC][-1]
        final_vals_list.append(current_final_val) # For table average

        if lower_is_better:
            if current_final_val < best_val:
                best_val, best_run_key, best_run_hist = current_final_val, exp, hist[METRIC]
            if current_final_val > worst_val:
                worst_val, worst_run_key, worst_run_hist = current_final_val, exp, hist[METRIC]
        else:
            if current_final_val > best_val:
                best_val, best_run_key, best_run_hist = current_final_val, exp, hist[METRIC]
            if current_final_val < worst_val:
                worst_val, worst_run_key, worst_run_hist = current_final_val, exp, hist[METRIC]
        # --- End Update best/worst ---

    # --- End Loop ---


# --- NEW LaTeX Table Generation Logic (4 Rows, 5 Columns) ---
    if print_metrics:
        print(f"% LaTeX Metrics Table for {title_prefix} - Metric: {METRIC} - Target Gen Group: {target_gen}")
        print("\\noindent\\textbf{" + f"{title_prefix}" + f" K={n_gen}" + "}") # Keep user's title marker

        sp_list = save_path.__str__().split("\\")
        abb_idx = sp_list.index("abb")
        print("\\begin{figure}[htbp]")
        print("\t\\centering")
        print("\t\\includegraphics[width=.85\\textwidth]{" + f"{'/'.join(sp_list[abb_idx:])}" + "}") 
        print("\t\\label{" + f"fig:app_strat_class_performance_{title_prefix.lower().replace(' ', '_').replace(':', '')}_{METRIC}_{target_gen}" + "}")        
        print("\\end{figure}")
        
        
        print("\\begin{table}[H]")
        print("\t\\vspace{-1em}")
        print("\t\\centering")
        # Define 5 columns as per user's tabular environment
        print("\t\\begin{tabular}{|c|c|c|c|}")
        print("\t\t\\hline")
        # Define header text matching the 5 columns, based on user's text header
        print(f"\t\tRun Type & Experiment & {metric_map[METRIC]} \\\\ \\hline") # Adjusted header text slightly

        metric_prints = [] # List to hold the four row strings
        metric_name = metric_map.get(METRIC, METRIC) # Get display name

        # --- Best Row ---
        n_gen_b, used_gen_b, best_val_str = "-", "-", "N/A" # Defaults
        if best_run_key: # Check if a best run was found
            meta_b = meta_info_copy.get(best_run_key, {})
            n_gen_b = meta_b.get('n_gen', '-') # Use '-' if meta key is missing
            used_gen_b = meta_b.get('used_gen', '-') # Use '-' if meta key is missing
            n_real_b = meta_b.get('n_real', '-') # Use '-' if meta key is missing
            n_fake_b = meta_b.get('n_fake', '-') # Use '-' if meta key is missing
            best_val_str = f"${best_val:.4f}$"
        metric_prints.append(f"\t\tbest & " + "\\(G_{" f"{n_gen_b}, {used_gen_b}" + "}\\)," + f" r:{n_real_b}, f:{n_fake_b}" + f" & {best_val_str}")

        # --- Worst Row ---
        n_gen_w, used_gen_w, worst_val_str = "-", "-", "N/A" # Defaults
        if worst_run_key: # Check if a worst run was found
            meta_w = meta_info_copy.get(worst_run_key, {})
            n_gen_w = meta_w.get('n_gen', '-') # Use '-' if meta key is missing
            used_gen_w = meta_w.get('used_gen', '-') # Use '-' if meta key is missinga
            n_real_w = meta_w.get('n_real', '-') # Use '-' if meta key is missing
            n_fake_w = meta_w.get('n_fake', '-') # Use '-' if meta key is missing
            worst_val_str = f"${worst_val:.4f}$"
        metric_prints.append(f"\t\tworst & " + "\\(G_{" f"{n_gen_w}, {used_gen_w}" + "}\\)," + f" r:{n_real_w}, f:{n_fake_w}" + f" & {worst_val_str}")

        # --- Median Row ---
        median_val_str = "N/A" # Default
        if final_vals_list: # Check if list has values
            try:
                # Use nanmedian for robustness against potential NaNs if data loading had issues
                median_final_val = np.nanmedian(final_vals_list)
                median_val_str = f"${median_final_val:.4f}$"
            except Exception as e:
                if verbose: print(f"Could not calculate median final value: {e}")
                median_val_str = "Error"
        metric_prints.append(f"\t\tmedian & - & {median_val_str}") # Use '-' for n-gen/n-used

        # --- Average Row ---
        avg_val_str = "N/A" # Default
        if final_vals_list: # Check if list has values
            try:
                avg_final_val = np.mean(final_vals_list)
                avg_val_str = f"${avg_final_val:.4f}$"
            except Exception as e:
                if verbose: print(f"Could not calculate average final value: {e}")
                avg_val_str = "Error"
        metric_prints.append(f"\t\taverage & - & {avg_val_str}") # Use '-' for n-gen/n-used

        # Print the rows joined by the required LaTeX line breaks
        print("\\\\ \\hline\n".join(metric_prints))
        # Print the final closing hline based on user example structure
        print("\t\t\\\\ \\hline")

        # Print table footer
        print("\t\\end{tabular}")
        # Optional: Add caption
        # print(f"\t\\caption{{Performance Summary for {metric_name.replace('_',' ')} across runs with target N-Gen group {target_gen}.}}")
        print("\\end{table}")
        print(f"% End LaTeX Table for {title_prefix}")
    # --- End LaTeX Table Generation ---


    # --- Calculate Epoch-wise Average/Median for Plotting (remains the same) ---
    if not all_vals:
        if verbose:
            print(f"Info: No valid non-baseline data found for metric '{METRIC}' after filtering. Plot may only show baseline.")
        # Still proceed to plot baseline if available, handled later
    avg, med = None, None # Initialize
    try:
        # Ensure all elements in all_vals are valid iterables of numbers
        valid_vals = [v for v in all_vals if hasattr(v, '__iter__') and all(isinstance(x, (int, float)) for x in v)]
        if valid_vals: # Proceed only if there's valid data
            max_len = max(map(len, valid_vals))
            # Pad with NaN for correct nanmean/nanmedian calculation
            all_vals_padded = [np.pad(np.array(a, dtype=float), (0, max_len - len(a)), mode='constant', constant_values=np.nan) for a in valid_vals]
            # Calculate mean/median ignoring NaNs
            with warnings.catch_warnings(): # Suppress potential "mean of empty slice" warning if a column is all NaNs
                 warnings.simplefilter("ignore", category=RuntimeWarning)
                 avg = np.nanmean(all_vals_padded, axis=0)
                 med = np.nanmedian(all_vals_padded, axis=0)
        elif verbose and not baseline_exp_key: # Only warn if there's truly nothing to plot
             print(f"Warning: No valid numerical data found in any runs for metric '{METRIC}'. Cannot plot avg/median/best/worst.")

    except ValueError as e:
        if verbose:
            print(f"Error during epoch-wise padding or aggregation: {e}. Check data types in metric lists.")
        # avg, med remain None
    # --- End Epoch-wise Calculation ---


    # --- Plotting Lines (Best, Worst, Avg, Median, Baseline) ---
    color_worst = '#D55E00'
    color_best = '#009E73'
    color_avg = '#0072B2'
    color_med = '#E69F00'
    color_base = '#000000'

    # Plot Worst Run
    if worst_run_key and worst_run_hist:
        meta_w = meta_info_copy.get(worst_run_key, {}) # Meta should exist from checks above
        used_gen_w = meta_w.get('used_gen', 'N/A')
        n_real_w = meta_w.get('n_real', 'N/A')
        n_fake_w = meta_w.get('n_fake', 'N/A')
        n_gen_w = meta_w.get('n_gen', 'N/A') # Get n_gen for label consistency
        worst_label = f"Worst Run (Final: {worst_val:.4f}), G=({n_gen_w},{used_gen_w}), R={n_real}, F={n_fake_w}"
        plt.plot(worst_run_hist, color=color_worst, linewidth=2, label=worst_label)
    elif len(final_vals_list) > 0 and verbose: # Check if runs existed but maybe failed best/worst logic
        print("Warning: Could not plot the worst run line, although non-baseline data exists.")

    # Plot Best Run
    if best_run_key and best_run_hist:
        meta_b = meta_info_copy.get(best_run_key, {}) # Meta should exist
        used_gen_b = meta_b.get('used_gen', 'N/A')
        n_real_b = meta_b.get('n_real', 'N/A')
        n_fake_b = meta_b.get('n_fake', 'N/A')
        n_gen_b = meta_b.get('n_gen', 'N/A') # Get n_gen for label consistency
        best_label = f"Best Run (Final: {best_val:.4f}), G=({n_gen_b},{used_gen_b}), R={n_real_b}, F={n_fake_b}"
        plt.plot(best_run_hist, color=color_best, linewidth=2, label=best_label)
    elif len(final_vals_list) > 0 and verbose:
        print("Warning: Could not plot the best run line, although non-baseline data exists.")

    # Plot Average Epoch-wise Line
    if avg is not None and not np.all(np.isnan(avg)): # Check if avg was calculated and is not all NaNs
        plt.plot(avg, color=color_avg, linewidth=2, label="Average (excl. baseline)")

    # Plot Median Epoch-wise Line
    if med is not None and not np.all(np.isnan(med)): # Check if med was calculated and is not all NaNs
        plt.plot(med, color=color_med, linewidth=2, linestyle='--', label="Median (excl. baseline)")

    # Plot Baseline Run
    if baseline_exp_key and baseline_exp_key in histories_copy and METRIC in histories_copy[baseline_exp_key] and histories_copy[baseline_exp_key][METRIC]:
        # Ensure baseline data is plottable
        baseline_hist_data = histories_copy[baseline_exp_key][METRIC]
        if baseline_hist_data and hasattr(baseline_hist_data, '__len__') and len(baseline_hist_data) > 0 and all(isinstance(x, (int, float)) for x in baseline_hist_data):
             plt.plot(baseline_hist_data, color=color_base, linewidth=2.5, linestyle=':', label=f'Baseline (Final: {round(baseline_hist_data[-1], 3)}, R={baseline_criteria_real}, F={baseline_criteria_fake})')
        elif verbose:
             print(f"Warning: Baseline experiment '{baseline_exp_key}' found but its metric data for '{METRIC}' is empty or invalid. Not plotting baseline.")
    # --- End Plotting Lines ---


    # --- Final Plot Setup (Title, Labels, Legend, Grid, Save/Show) ---
    # Determine dataset name for title
    plot_dataset_name = "Unknown Dataset"
    try:
        # Prioritize non-baseline runs for dataset name
        first_valid_key = next((k for k, m in meta_info_copy.items() if k != baseline_exp_key and 'dataset' in m), None)
        if first_valid_key:
            plot_dataset_name = meta_info_copy[first_valid_key].get('dataset', 'N/A')
        # Fallback to baseline if no other runs had dataset info
        elif baseline_exp_key and baseline_exp_key in meta_info_copy and 'dataset' in meta_info_copy[baseline_exp_key]:
            plot_dataset_name = meta_info_copy[baseline_exp_key].get('dataset', 'N/A')
    except Exception as e:
        if verbose:
            print(f"Could not reliably determine dataset name for title: {e}")

    # Create title
    metric_title = metric_map.get(METRIC, METRIC.replace('_', ' ').title())
    plt.title(f"{title_prefix} {metric_title} - Dataset: {plot_dataset_name}, {title_suffix}, N-Gen Group: {target_gen}")
    plt.xlabel("Epoch")
    plt.ylabel(metric_title) # Use consistent naming

    # Legend handling
    handles, labels = plt.gca().get_legend_handles_labels()
    if labels: # Only show legend if there are labeled lines
        # Create a mapping from label text to handle to remove duplicate labels if any arise
        by_label = dict(zip(labels, handles))
        plt.legend(by_label.values(), by_label.keys(), fontsize='small', loc='best')

    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout() # Adjust layout

    # Save or Show Plot
    if save_path: # Check if save_path is provided
        try:
            save_path.parent.mkdir(parents=True, exist_ok=True)
            plt.savefig(save_path, bbox_inches='tight', dpi=200)
            if verbose:
                print(f"Plot saved to {save_path}")
        except Exception as e:
            if verbose:
                print(f"Error saving plot to {save_path}: {e}")
            # Decide if we should still show the plot if saving fails
            if show:
                 if verbose: print("Saving failed, attempting to display plot...")
                 plt.show()
    elif show: # Only show if save_path is not given and show is True
        if verbose:
            print("Displaying plot...")
        plt.show()

    plt.close() # Close the figure to free memory

### NOTE: 
there are only experiments with 3, 5, 7, 10 generators 

In [5]:
# load all histories: 

experiments_by_used_gen = {}

histories = {}
meta_info = {}
folders =[
    'MNIST_STRATIFIED_CLASSIFIERS_MADGAN_NEW',
    'MNIST_STRATIFIED_CLASSIFIERS_cMADGAN_NEW',
    'FASHION_STRATIFIED_CLASSIFIERS_MADGAN_NEW',
    'FASHION_STRATIFIED_CLASSIFIERS_cMADGAN_NEW'
]
suffix_map = {
    'MNIST_STRATIFIED_CLASSIFIERS_MADGAN_NEW':"MADGAN",
    'MNIST_STRATIFIED_CLASSIFIERS_cMADGAN_NEW':"cMADGAN",
    'FASHION_STRATIFIED_CLASSIFIERS_MADGAN_NEW':"MADGAN",
    'FASHION_STRATIFIED_CLASSIFIERS_cMADGAN_NEW':"cMADGAN"
}

for FOLDER in folders:
    DATASET, _, _, TYPE, _ = FOLDER.split('_')

    print("\\subsubsection{Dataset: " + DATASET + ", Architecture: " + TYPE.replace('_', ' ') + "}")

    metrics = ["val_f1_score",]   
    base_path = Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/notebooks")
    strat_exp_path = base_path / 'experiments' / FOLDER 
    experiments = os.listdir(strat_exp_path)

    expansen_experiments = []
    replacement_experiments = []

    for exp in experiments: 
        if "real_5000_gen_0" in exp: 
            expansen_experiments.append(exp)
            replacement_experiments.append(exp)
        elif "real_5000" in exp: 
            expansen_experiments.append(exp)
        else:
            try:
                if int(exp.split("_")[-3]) < 5000:
                    replacement_experiments.append(exp)
            except Exception:
                pass

    for METRIC in metrics:
        for target_gen in [3, 5, 7, 10]:

            for exp in expansen_experiments: 
                meta = extract_info_from_experiment_name(exp)
                if meta['n_gen'] == target_gen:
                    try:
                        history = np.load(Path(strat_exp_path) / exp / 'training_history.npy', allow_pickle=True).item()
                        histories[exp] = history
                        meta_info[exp] = meta
                    except Exception:
                        pass
            
            histories = sort_dict_based_on_n_real_images(histories)
            meta_info = sort_dict_based_on_n_real_images(meta_info)

            if histories:
                plot_history_strat_classifiers(
                    histories, 
                    meta_info, 
                    Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/latex/master_thesis/abb/strat_classifier_performance") / FOLDER / "expansion_experiments" / f"{METRIC}_{TYPE}_{DATASET}_n_gen_{target_gen}_all.png",    
                    target_gen,
                    METRIC, 
                    title_prefix="Expansion Exp.:",
                    title_suffix=f"Type: {suffix_map[FOLDER]}",
                    print_metrics=True,
                )
            histories = {}
            meta_info = {}

            for exp in replacement_experiments: 
                meta = extract_info_from_experiment_name(exp)
                if meta['n_gen'] == target_gen:
                    try:
                        history = np.load(Path(strat_exp_path) / exp / 'training_history.npy', allow_pickle=True).item()
                        histories[exp] = history
                        meta_info[exp] = meta
                    except Exception:
                        pass

            histories = sort_dict_based_on_n_real_images(histories)
            meta_info = sort_dict_based_on_n_real_images(meta_info)

            if histories:
                plot_history_strat_classifiers(
                    histories, 
                    meta_info, 
                    Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/latex/master_thesis/abb/strat_classifier_performance") / FOLDER / "replacement_experiments" / f"{METRIC}_{TYPE}_{DATASET}_n_gen_{target_gen}_all.png",    
                    target_gen,
                    METRIC,
                    title_prefix="Replacement Exp.:", 
                    title_suffix=f"Type: {suffix_map[FOLDER]}",
                    print_metrics=True,
                )
                
            print("\\newpage")
                
            histories = {}
            meta_info = {}

\subsubsection{Dataset: MNIST, Architecture: MADGAN}
% LaTeX Metrics Table for Expansion Exp.: - Metric: val_f1_score - Target Gen Group: 3
\noindent\textbf{Expansion Exp.: K=3}
\begin{figure}[htbp]
	\centering
	\includegraphics[width=.85\textwidth]{abb/strat_classifier_performance/MNIST_STRATIFIED_CLASSIFIERS_MADGAN_NEW/expansion_experiments/val_f1_score_MADGAN_MNIST_n_gen_3_all.png}
	\label{fig:app_strat_class_performance_expansion_exp._val_f1_score_3}
\end{figure}
\begin{table}[H]
	\vspace{-1em}
	\centering
	\begin{tabular}{|c|c|c|c|}
		\hline
		Run Type & Experiment & Val F1 \\ \hline
		best & \(G_{3, 0}\), r:5000, f:1000 & $0.9899$\\ \hline
		worst & \(G_{3, 0}\), r:5000, f:5000 & $0.9841$\\ \hline
		median & - & $0.9864$\\ \hline
		average & - & $0.9867$
		\\ \hline
	\end{tabular}
\end{table}
% End LaTeX Table for Expansion Exp.:
% LaTeX Metrics Table for Replacement Exp.: - Metric: val_f1_score - Target Gen Group: 3
\noindent\textbf{Replacement Exp.: K=3}
\begin{figure}[htbp]
	\

In [6]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import warnings
import time # Added for timestamp

# Define metric_map globally or ensure it's accessible
metric_map = {
    "val_f1_score": "Val F1",
    "val_loss": "Val Loss",
    "val_accuracy": "Val Acc."
}

def plot_history_simple(
    histories: dict,
    meta_info: dict,
    save_path: Path,
    metric,
    title_prefix,
    title_suffix,
    dataset = None,
    show: bool = False,
    # Add print_metrics and verbose parameters
    print_metrics: bool = False,
    verbose: bool = False
) -> None:
    """
    Simplified version of plotting metric history with baseline, best, worst, average, and median.
    Includes LaTeX table output option.

    Args:
        histories (dict): Dict of history objects {exp_key: {'metric': [values]}}.
        meta_info (dict): Dict {exp_key: {'n_real': N, 'n_fake': M, 'dataset': 'name', 'n_gen': G, 'n_used': U}}.
        save_path (Path): Path to save plot. Can be None if show is True.
        metric (str): Metric to plot (e.g., 'val_accuracy').
        title_prefix (str): Prefix for the plot title.
        title_suffix (str): Suffix for the plot title.
        dataset (str, optional): Override dataset name. Defaults to None.
        show (bool): If True, display plot instead of saving. Ignored if save_path is provided.
        print_metrics (bool): If True, print summary LaTeX table.
        verbose (bool): If True, print warnings and info messages.
    """
    # Make copies to avoid modifying original dictionaries
    histories_copy = histories.copy()
    meta_info_copy = meta_info.copy()

    plt.figure(figsize=(10, 6))

    # --- Baseline Identification ---
    baseline_key = None
    # Define baseline criteria (adjust if needed)
    baseline_criteria_real, baseline_criteria_fake = 5000, 0 # Example criteria
    for k, v in meta_info_copy.items():
        # Check if key exists in histories as well before considering meta
        if k in histories_copy:
            try:
                # Use .get with default to avoid errors if keys missing
                # Ensure comparison is done with integers
                n_real = int(v.get('n_real', -1))
                n_fake = int(v.get('n_fake', -1))
                if n_real == baseline_criteria_real and n_fake == baseline_criteria_fake:
                    baseline_key = k
                    if verbose: print(f"Identified baseline experiment: '{baseline_key}'")
                    break # Found first baseline match
            except (ValueError, TypeError):
                 if verbose: warnings.warn(f"Could not parse n_real/n_fake for potential baseline '{k}'. Skipping.")
                 continue # Skip this key if parsing fails
    if not baseline_key and verbose:
        print(f"Warning: No baseline experiment found matching R={baseline_criteria_real}, F={baseline_criteria_fake}.")
    # --- End Baseline Identification ---

    # --- Initialization for Best/Worst/Aggregation ---
    all_vals = []       # Stores full history lists for epoch-wise avg/median plot lines
    final_vals_list = [] # Stores only the *final* value of the metric for table stats
    best_key, worst_key = None, None
    best_hist, worst_hist = None, None
    lower_is_better = 'loss' in metric.lower() # Determine optimization direction

    # Initialize best/worst values correctly based on optimization direction
    if lower_is_better:
        best_val = float('inf')
        worst_val = float('-inf')
    else: # Higher is better (e.g., accuracy, F1)
        best_val = float('-inf')
        worst_val = float('inf')
    # --- End Initialization ---


    # --- Loop through experiments to find best/worst, plot faint lines, collect values ---
    for k, hist in histories_copy.items():
        # --- Validate data for this run ---
        if metric not in hist or not hist[metric] or not hasattr(hist[metric], '__len__') or len(hist[metric]) == 0:
            if verbose: print(f"Verbose: Skipping run '{k}': Metric '{metric}' missing, empty, or invalid.")
            continue
        # Ensure data is numeric and not empty before proceeding
        vals = hist[metric]
        if not all(isinstance(x, (int, float)) for x in vals):
            if verbose: print(f"Verbose: Skipping run '{k}': Non-numeric data found in metric '{metric}'.")
            continue
        # --- End Validation ---

        # Skip baseline run for aggregation and faint plots
        if k == baseline_key:
            continue

        # Plot faint line for non-baseline runs
        plt.plot(vals, color=(0.6, 0.6, 0.6, 0.3))

        # Collect data for calculations
        try:
            final_val = float(vals[-1]) # Ensure it's a float for comparison
            all_vals.append(vals) # Keep full history for epoch-wise plot lines
            final_vals_list.append(final_val) # Collect final value for table stats

            # Update best/worst run info based on the final value
            if lower_is_better:
                if final_val < best_val:
                    best_val, best_key, best_hist = final_val, k, vals
                # Use '>=' for worst to handle ties consistently (e.g., pick last one encountered)
                if final_val >= worst_val:
                    worst_val, worst_key, worst_hist = final_val, k, vals
            else: # Higher is better
                if final_val > best_val:
                    best_val, best_key, best_hist = final_val, k, vals
                # Use '<=' for worst
                if final_val <= worst_val:
                    worst_val, worst_key, worst_hist = final_val, k, vals
        except (ValueError, TypeError, IndexError) as e:
             if verbose: print(f"Verbose: Skipping best/worst update for run '{k}': Error processing final value. {e}")
             continue # Skip if final value cannot be processed

    # --- End Loop ---


    # --- Generate LaTeX Table (if requested) ---
    if print_metrics:
        sp_list = save_path.__str__().split("\\")
        abb_idx = sp_list.index("abb")
        print(f"% LaTeX Metrics Table for {title_prefix} - Metric: {metric}")
        # Optional: print(title_prefix) # Uncomment if needed as part of text output
        
        print(f"% LaTeX Metrics Table for {title_prefix} - Metric: {METRIC} - Target Gen Group: {target_gen}")
        print("\\noindent\\textbf{" + f"{title_prefix}" + f"" + "}") # Keep user's title marker
        
        print("\\begin{figure}[htbp]")
        print("\t\\centering")
        print("\t\\includegraphics[width=.85\\textwidth]{" + f"{'/'.join(sp_list[abb_idx:])}" + "}") 
        print("\t\\label{" + f"fig:app_strat_class_performance_{title_prefix.lower().replace(' ', '_').replace(':', '')}_{METRIC}_" + "}")        
        print("\\end{figure}")
        
        print("\\begin{table}[H]")
        print("\t\\centering")
        print("\t\\vspace{-1em}")
        # Define 5 columns: Run Type, Metric, N-Gen, N-Used, Performance
        print("\t\\begin{tabular}{|c|c|c|c|c|}")
        print("\t\t\\hline")
        # Header text matching the 5 columns
        print("\t\tRun Type & Metric & n-gen & n-used & Performance \\\\ \\hline")

        metric_prints = [] # List to hold the four row strings
        # Use global metric_map or default formatting
        metric_name = metric_map.get(metric, metric.replace('_', ' ').title())

        # --- Best Row ---
        n_gen_b, used_gen_b, best_val_str = "-", "-", "N/A" # Defaults
        if best_key: # Check if a best run was identified
            meta_b = meta_info_copy.get(best_key, {}) # Safely get meta info
            n_gen_b = meta_b.get('n_gen', '-') # Use '-' if meta key is missing
            used_gen_b = meta_b.get('used_gen', '-') # Use '-' if meta key is missing
            best_val_str = f"${best_val:.4f}$" # Format the value found during loop
        metric_prints.append(f"\t\tbest & {metric_name} & {n_gen_b} & {used_gen_b} & {best_val_str}")

        # --- Worst Row ---
        n_gen_w, used_gen_w, worst_val_str = "-", "-", "N/A" # Defaults
        if worst_key: # Check if a worst run was identified
            meta_w = meta_info_copy.get(worst_key, {}) # Safely get meta info
            n_gen_w = meta_w.get('n_gen', '-')
            used_gen_w = meta_w.get('used_gen', '-')
            worst_val_str = f"${worst_val:.4f}$" # Format the value found during loop
        metric_prints.append(f"\t\tworst & {metric_name} & {n_gen_w} & {used_gen_w} & {worst_val_str}")

        # --- Median Row ---
        median_val_str = "N/A" # Default
        if final_vals_list: # Check if any final values were collected
            try:
                # Use nanmedian for robustness against potential NaNs
                median_final_val = np.nanmedian(final_vals_list)
                median_val_str = f"${median_final_val:.4f}$"
            except Exception as e:
                if verbose: print(f"Verbose: Could not calculate median final value: {e}")
                median_val_str = "Error"
        metric_prints.append(f"\t\tmedian & {metric_name} & - & - & {median_val_str}")

        # --- Average Row ---
        avg_val_str = "N/A" # Default
        if final_vals_list: # Check if any final values were collected
            try:
                avg_final_val = np.mean(final_vals_list)
                avg_val_str = f"${avg_final_val:.4f}$"
            except Exception as e:
                if verbose: print(f"Verbose: Could not calculate average final value: {e}")
                avg_val_str = "Error"
        metric_prints.append(f"\t\taverage & {metric_name} & - & - & {avg_val_str}")

        # Print the formatted rows joined by LaTeX line breaks and horizontal rules
        print("\\\\ \\hline\n".join(metric_prints))
        print("\t\t\\\\ \\hline") # Final horizontal line

        # Print table footer
        print("\t\\end{tabular}")
        # Optional: Add caption
        # caption_metric_name = metric_name.replace('_',' ') # Make metric name readable for caption
        # print(f"\t\\caption{{Performance Summary for {caption_metric_name}}}")
        print("\\end{table}")
        print(f"% End LaTeX Table for {title_prefix}")
    # --- End LaTeX Table Generation ---


    # --- Compute epoch-wise avg/median for plot lines ---
    avg, med = None, None # Initialize
    if all_vals: # Check if there are any non-baseline runs to average/median over epochs
        try:
            # Filter ensure all runs are valid lists/arrays of numbers before padding
            valid_vals = [v for v in all_vals if hasattr(v, '__iter__') and all(isinstance(x, (int, float)) for x in v)]
            if valid_vals:
                max_len = max(len(v) for v in valid_vals)
                # Pad with NaN for correct nanmean/nanmedian calculation across epochs
                padded = [np.pad(np.array(v, dtype=float), (0, max_len - len(v)), mode='constant', constant_values=np.nan) for v in valid_vals]
                # Suppress RuntimeWarning for slices with all NaNs
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore", category=RuntimeWarning)
                    avg = np.nanmean(padded, axis=0)
                    med = np.nanmedian(padded, axis=0)
            elif verbose: print("Verbose: No valid runs found for epoch-wise averaging/median.")
        except Exception as e:
            if verbose: print(f"Verbose: Error calculating epoch-wise avg/median: {e}")
            avg, med = None, None # Ensure they are None on error
    # --- End epoch-wise calculation ---


    # --- Plot special curves (Baseline, Best, Worst, Avg, Median) ---
    # Define CUD color palette
    color_worst = '#D55E00' # Rust
    color_best = '#009E73'  # Teal
    color_avg = '#0072B2'  # Blue
    color_med = '#E69F00'  # Golden
    color_base = '#000000' # Black

    # Plot Baseline
    if baseline_key and baseline_key in histories_copy and metric in histories_copy[baseline_key]:
         baseline_hist_data = histories_copy[baseline_key][metric]
         # Check if baseline data is valid before plotting
         if baseline_hist_data and hasattr(baseline_hist_data, '__len__') and len(baseline_hist_data) > 0 and all(isinstance(x, (int, float)) for x in baseline_hist_data):
             plt.plot(baseline_hist_data, color=color_base, linestyle=':', linewidth=2.5,
                      label=f'Baseline (Final: {round(baseline_hist_data[-1], 3)}, R={baseline_criteria_real}, F={baseline_criteria_fake})')
         elif verbose: print(f"Verbose: Baseline '{baseline_key}' metric '{metric}' data invalid, not plotting.")

    # Plot Best Run (using metadata looked up previously for label)
    if best_key and best_hist:
        meta_b = meta_info_copy.get(best_key, {})
        n_gen_b = meta_b.get('n_gen', '?') # Use '?' if missing meta
        n_used_b = meta_b.get('used_gen', '?')
        n_real_b = meta_b.get('n_real', '?')
        n_fake_b = meta_b.get('n_fake', '?')
        # Use the formatted best_val string (remove $ for label) or raw value
        best_label_val = best_val_str.strip('$') if best_key else 'N/A'
        if "TDA" in title_suffix:
            best_label = f"Best Run (Final: {best_label_val}), R={n_real_b}, F={n_fake_b}"
        else:
            best_label = f"Best Run (Final: {best_label_val}), G=({n_gen_b},{n_used_b}), R={n_real_b}, F={n_fake_b}"
        plt.plot(best_hist, color=color_best, linewidth=2, label=best_label)
    elif len(final_vals_list) > 0 and verbose: # Check if runs existed but maybe failed best/worst logic
        print("Verbose: Could not plot the best run line, although non-baseline data exists.")


    # Plot Worst Run (using metadata looked up previously for label)
    if worst_key and worst_hist:
        meta_w = meta_info_copy.get(worst_key, {})
        n_gen_w = meta_w.get('n_gen', '?')
        n_used_w = meta_w.get('used_gen', '?')
        n_real_w = meta_w.get('n_real', '?')
        n_fake_w = meta_w.get('n_fake', '?')
        # Use the formatted worst_val string (remove $ for label) or raw value
        worst_label_val = worst_val_str.strip('$') if worst_key else 'N/A'
        if "TDA" in title_suffix:
            worst_label = f"Worst Run (Final: {worst_label_val}), R={n_real_w}, F={n_fake_w}"
        else: 
            worst_label = f"Worst Run (Final: {worst_label_val}), G=({n_gen_w},{n_used_w}), R={n_real_w}, F={n_fake_w}"
        plt.plot(worst_hist, color=color_worst, linewidth=2, label=worst_label)
    elif len(final_vals_list) > 0 and verbose:
        print("Verbose: Could not plot the worst run line, although non-baseline data exists.")

    # Plot Average Epoch-wise Line
    if avg is not None and not np.all(np.isnan(avg)): # Check avg exists and is not all NaNs
        plt.plot(avg, color=color_avg, linewidth=2, label='Average (excl. baseline)')

    # Plot Median Epoch-wise Line
    if med is not None and not np.all(np.isnan(med)): # Check med exists and is not all NaNs
        plt.plot(med, color=color_med, linewidth=2, linestyle='--', label='Median (excl. baseline)')
    # --- End Plotting ---


    # --- Final Plot Setup (Title, Labels, Legend, Grid) ---
    # Determine dataset name (improved logic)
    if not dataset: # Try to determine dataset if not provided
         ds_keys = [k for k in meta_info_copy if k in histories_copy] # Keys with both meta and history
         # Prefer baseline dataset name if available
         if baseline_key and 'dataset' in meta_info_copy.get(baseline_key, {}):
             dataset = meta_info_copy[baseline_key].get('dataset', 'Unknown')
         # Fallback to first key with dataset info if baseline has none or doesn't exist
         elif ds_keys:
             dataset_found = False
             for k in ds_keys:
                 ds_name = meta_info_copy[k].get('dataset')
                 if ds_name:
                     dataset = ds_name
                     dataset_found = True
                     break
             if not dataset_found: dataset = 'Unknown' # If no run has dataset info
         else:
             dataset = 'Unknown' # If no runs with meta/history

    # Set title, labels using metric map
    metric_title = metric_map.get(metric, metric.replace('_', ' ').title())
    plt.title(f"{title_prefix} {metric_title} - Dataset: {dataset}, {title_suffix}")
    plt.xlabel("Epoch")
    plt.ylabel(metric_title)
    plt.grid(True, linestyle='--', alpha=0.6)

    # Legend handling (remove duplicates automatically using dictionary)
    handles, labels = plt.gca().get_legend_handles_labels()
    if labels: # Only show legend if there are labeled lines
        by_label = dict(zip(labels, handles)) # dict removes duplicate labels automatically
        plt.legend(by_label.values(), by_label.keys(), fontsize='small', loc='best')

    plt.tight_layout() # Adjust layout to prevent overlap
    # --- End Plot Setup ---


    # --- Save or Show Plot ---
    if save_path: # Prefer saving if path is given
        try:
            save_path.parent.mkdir(parents=True, exist_ok=True) # Ensure directory exists
            plt.savefig(save_path, bbox_inches='tight', dpi=200)
            # Use standard print for user feedback, regardless of verbose
            if verbose:
                print(f"Plot saved to {save_path}")
        except Exception as e:
            print(f"Error saving plot to {save_path}: {e}")
            # Optionally show if saving fails and show is True
            if show:
                 print("Saving failed, attempting to display plot...")
                 plt.show()
    elif show: # Only show if save_path is not given and show is True
        if verbose: print("Displaying plot...")
        plt.show()

    plt.close() # Close the plot figure to free up memory
    # --- End Save or Show ---

In [7]:
metrics = ["val_f1_score"]   

base_path = Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/notebooks")
strat_exp_path = base_path / 'experiments'  
suffix_map = {
    'FASHION_STRATIFIED_CLASSIFIERS_COND_GAN':"Conditional",
    'FASHION_STRATIFIED_CLASSIFIERS_VANILLA_GAN':"Deep Conv.",
    'MNIST_STRATIFIED_CLASSIFIERS_COND_GAN':"Conditional",
    'MNIST_STRATIFIED_CLASSIFIERS_VANILLA_GAN':"Deep Conv."
}

for folder in [
    "MNIST_STRATIFIED_CLASSIFIERS_VANILLA_GAN",
    "MNIST_STRATIFIED_CLASSIFIERS_COND_GAN",
    "FASHION_STRATIFIED_CLASSIFIERS_VANILLA_GAN",
    "FASHION_STRATIFIED_CLASSIFIERS_COND_GAN",
]: 
    strat_exp_path = base_path / 'experiments' / folder 
    experiments = os.listdir(strat_exp_path)
    histories = {}
    meta_info = {}

    DATASET = folder.split("_")[0]
    TYPE = " ".join(folder.split("_")[3:])

    expansen_experiments = []
    replacement_experiments = []

    print("\\subsubsection{Dataset: " + DATASET + ", Architecture: " + TYPE.replace('_', ' ') + "}")

    
    for exp in experiments: 
        if "real_5000_gen_0" in exp: 
            expansen_experiments.append(exp)
            replacement_experiments.append(exp)
        elif "real_5000" in exp: 
            expansen_experiments.append(exp)
        else:
            try:
                if int(exp.split("_")[-3]) < 5000:
                    replacement_experiments.append(exp)
            except Exception:
                pass


    for METRIC in metrics:
        print(f"CURRENT METRIC: {METRIC.replace('_', ' ')}")    

        histories = {}
        meta_info = {}
        
        for exp in expansen_experiments: 
            meta = extract_info_from_experiment_name(exp)
            try:
                history = np.load(Path(strat_exp_path) / exp / 'training_history.npy', allow_pickle=True).item()
                histories[exp] = history
                meta_info[exp] = meta
            except Exception as e:
                print("1111111111111ERROROROEROEOREOROEROE")
                print( e)
                pass
        histories = sort_dict_based_on_n_real_images(histories)
        meta_info = sort_dict_based_on_n_real_images(meta_info)

        plot_history_simple(
            histories, meta_info, metric=METRIC, 
            print_metrics=True,
            title_prefix="Expansion Exp.:", 
            title_suffix=f"Type: {suffix_map[folder]}",
            save_path= Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/latex/master_thesis/abb/strat_classifier_performance") / folder / "expansion_experiments" / f"{METRIC}_{folder.split('_')[-2:-1]}_{DATASET}_all.png",    
        )

        histories = {}
        meta_info = {}
        
        for exp in replacement_experiments: 
            meta = extract_info_from_experiment_name(exp)
            try: 
                history = np.load(Path(strat_exp_path) / exp / 'training_history.npy', allow_pickle=True).item()
                histories[exp] = history
                meta_info[exp] = meta
            except Exception as e:
                print("2222222222222ERROROROEROEOREOROEROE")
                print( e)
                pass
        histories = sort_dict_based_on_n_real_images(histories)

        
        meta_info = sort_dict_based_on_n_real_images(meta_info)

        plot_history_simple(
            histories, meta_info, metric=METRIC, 
            print_metrics=True,
            title_prefix="Replacement Exp.:", 
            title_suffix=f"Type: {suffix_map[folder]}",
            save_path= Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/latex/master_thesis/abb/strat_classifier_performance") / folder / "replacement_experiments" / f"{METRIC}_{folder.split('_')[-2:-1]}_{DATASET}_all.png",    
        )        
        print("\\newpage")

\subsubsection{Dataset: MNIST, Architecture: VANILLA GAN}
CURRENT METRIC: val f1 score
% LaTeX Metrics Table for Expansion Exp.: - Metric: val_f1_score
% LaTeX Metrics Table for Expansion Exp.: - Metric: val_f1_score - Target Gen Group: 10
\noindent\textbf{Expansion Exp.:}
\begin{figure}[htbp]
	\centering
	\includegraphics[width=.85\textwidth]{abb/strat_classifier_performance/MNIST_STRATIFIED_CLASSIFIERS_VANILLA_GAN/expansion_experiments/val_f1_score_['VANILLA']_MNIST_all.png}
	\label{fig:app_strat_class_performance_expansion_exp._val_f1_score_}
\end{figure}
\begin{table}[H]
	\centering
	\vspace{-1em}
	\begin{tabular}{|c|c|c|c|c|}
		\hline
		Run Type & Metric & n-gen & n-used & Performance \\ \hline
		best & Val F1 & 0 & 0 & $0.9884$\\ \hline
		worst & Val F1 & 0 & 0 & $0.9837$\\ \hline
		median & Val F1 & - & - & $0.9852$\\ \hline
		average & Val F1 & - & - & $0.9857$
		\\ \hline
	\end{tabular}
\end{table}
% End LaTeX Table for Expansion Exp.:
% LaTeX Metrics Table for Replacement Exp

In [8]:
metrics = ["val_f1_score",]   

base_path = Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/notebooks")
strat_exp_path = base_path / 'experiments/TDA_STRATIFIED_CLASSIFIER'  

for folder in [
    "tda_mnist",
    "tda_fashion_mnist", 
]: 
    strat_exp_path = base_path / 'experiments/TDA_STRATIFIED_CLASSIFIERS' / folder 
    experiments = os.listdir(strat_exp_path)
    histories = {}
    meta_info = {}

    dataset = "FASHION" if "fashion" in folder else "MNIST"
    TYPE = folder.split("_")[0]

    
    print("\\subsubsection{Dataset: " + dataset + ", Architecture: " + TYPE.replace('_', ' ') + "}")

    expansen_experiments = []
    replacement_experiments = []

    DATASET = folder.split("_")[1]
        
    for exp in experiments: 
        if "real_5000_gen_0" in exp: 
            expansen_experiments.append(exp)
            replacement_experiments.append(exp)
        elif "real_5000" in exp: 
            expansen_experiments.append(exp)
        else:
            try:
                if int(exp.split("_")[-3]) < 5000:
                    replacement_experiments.append(exp)
            except Exception:
                pass


    for METRIC in metrics:
        print(f"CURRENT METRIC: {METRIC.replace('_', ' ')}")    
        histories = {}
        meta_info = {}
        
        for exp in expansen_experiments: 
            meta = extract_info_from_experiment_name(exp)
            try:
                history = np.load(Path(strat_exp_path) / exp / 'training_history.npy', allow_pickle=True).item()
                histories[exp] = history
                meta_info[exp] = meta
            except Exception as e:
                print("1111111111111ERROROROEROEOREOROEROE")
                print( e)
                pass
        histories = sort_dict_based_on_n_real_images(histories)
        meta_info = sort_dict_based_on_n_real_images(meta_info)    
        plot_history_simple(
            histories, meta_info, metric=METRIC, 
            print_metrics=True,
            title_prefix="Expansion Exp.:", 
            title_suffix=f"Type: TDA",
            dataset=dataset, 
            save_path= Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/latex/master_thesis/abb/strat_classifier_performance") / folder / "expansion_experiments" / f"{METRIC}_{folder}_{DATASET}_all.png",    
        )    
        histories = {}
        meta_info = {}
        
        for exp in replacement_experiments: 
            meta = extract_info_from_experiment_name(exp)
            try: 
                history = np.load(Path(strat_exp_path) / exp / 'training_history.npy', allow_pickle=True).item()
                histories[exp] = history
                meta_info[exp] = meta
            except Exception as e:
                print("2222222222222ERROROROEROEOREOROEROE")
                print( e)
                pass
        histories = sort_dict_based_on_n_real_images(histories) 
        
        meta_info = sort_dict_based_on_n_real_images(meta_info)    
        plot_history_simple(
            histories, meta_info, metric=METRIC, 
            print_metrics=True,
            title_prefix="Replacement Exp.:", 
            title_suffix=f"Type: TDA",
            dataset=dataset, 
            save_path= Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/latex/master_thesis/abb/strat_classifier_performance") / folder / "replacement_experiments" / f"{METRIC}_{folder}_{DATASET}_all.png", 
        )        
        print("\\newpage")

\subsubsection{Dataset: MNIST, Architecture: tda}
CURRENT METRIC: val f1 score
% LaTeX Metrics Table for Expansion Exp.: - Metric: val_f1_score
% LaTeX Metrics Table for Expansion Exp.: - Metric: val_f1_score - Target Gen Group: 10
\noindent\textbf{Expansion Exp.:}
\begin{figure}[htbp]
	\centering
	\includegraphics[width=.85\textwidth]{abb/strat_classifier_performance/tda_mnist/expansion_experiments/val_f1_score_tda_mnist_mnist_all.png}
	\label{fig:app_strat_class_performance_expansion_exp._val_f1_score_}
\end{figure}
\begin{table}[H]
	\centering
	\vspace{-1em}
	\begin{tabular}{|c|c|c|c|c|}
		\hline
		Run Type & Metric & n-gen & n-used & Performance \\ \hline
		best & Val F1 & 0 & 0 & $0.9938$\\ \hline
		worst & Val F1 & 0 & 0 & $0.9923$\\ \hline
		median & Val F1 & - & - & $0.9934$\\ \hline
		average & Val F1 & - & - & $0.9932$
		\\ \hline
	\end{tabular}
\end{table}
% End LaTeX Table for Expansion Exp.:
% LaTeX Metrics Table for Replacement Exp.: - Metric: val_f1_score
% LaTeX Metrics